In [ ]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates csv for lexical access experiment lists
segFunctions.csv_from_excel('Lexical_Access_Experimental_Item_Setup.xlsx','All_LexExperiment_Lists','./data/original_data/exp_files/expLists_all.csv')

#creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')


In [ ]:
df = pd.read_csv('./data/temp_data/nonwords_all.csv')
i = 0
j = 9
nonwords = []
df.tail()

In [ ]:
dfTemp = df.loc[i:j,:]
#dfTemp.sort_values(by=['Rand'])
df_sorted = dfTemp.sort_values(by='Rand',ascending = True)
#print(len(dfTemp))
#dfTemp.dtypes
df_sorted


In [ ]:
token = df_sorted.iat[0,3]
print(token)

In [ ]:
print(nonwords)

In [ ]:
nonwords.append(token)
print(nonwords)

In [ ]:
nonwords = []
i = 0
j = 9
while j < 900:
    df_temp = df.loc[i:j,:]
    df_sorted = df_temp.sort_values(by='Rand',ascending = True)
    token = df_sorted.iat[0,3]
    nonwords.append(token)
    i += 10
    j += 10
print(nonwords)

In [ ]:
nonwords = []
i = 0
j = 9
k = 0
try:
    while j < 800:
        df_temp = df.loc[i:j,:]
        df_sorted = df_temp.sort_values(by='Rand',ascending = True)
        token = df_sorted.iat[k,3]
        print(token,i,j,k)
        if token in nonwords:
            k += 1
        else:
            nonwords.append(token)
            i += 10
            j += 10
            k = 0
except IndexError:
    print("Upper bound reached")
finally:
    print(nonwords)
    

In [ ]:
print(nonwords)

In [ ]:
print(j)

In [ ]:
print(len(nonwords))

# The Good Stuff
The code block below in this Jupyter Notebook takes the excel file containing nonwords created by the Wuggy.app application and generates a list of nonwords ready for the PyschoPy experiment.

## Wuggy.app
Critical items were taken from Carreiras and Perea (2002) experiment 4. All the critical items fell into one of two structures CV.CV.CV or CVC.CVC where 40 were of the first type and 40 were of the second type for a total of 80 critical items.

This list of 80 critical items were entered into Wuggy.app as words and the second column gave Wuggy.app the correction Spanish syllabification of each critical word—generated from within the application. I then used the verify feature to ensure all data was correct. This resulted in one error because "malena" was not found in the Wuggy.app dictionary. Therefore, I gave the correct syllabification of "ma-le-na" where hypens represent syllable boundaries. I then re-verified the data before runnning the analysis. Once I receive no errors, I ran the following analysis to generate new nonwords. 

### Parameters:

**General Settings:**
* Language module: Orthographic Spanish
* Output type: Only pseudowords
* Maximal number of candidates: 10 per word
* Maximal search time per word: 10 seconds
    
**Output Restrictions:**
* Checked: Match length of subsyllabic segements
* Checked: Match letter length
* Checked: Match transition frequencies (concentric search)
* Checked: Match subsyllabic segments: "2" out of "3"
        
**Output Options:**
* *Syllables* chosen from dropdown box
* Checked: Lexicality
* Checked: OLD20
* Checked: Neighbors at edit distance 1
* Checked: Number of overlapping segments
* Checked: Deviation statistics
        
### Output file

Wuggy outputs a file that contains all the raw data that was then imported into Microsoft Excel to preserve all the unicode characters contained in the data. **Do not simply open this from Windows Explorer or Finder** as it will replace all encoding of the file eliminating special characters.

**WARNING:** Wuggy uses syllabification and it removes the accented vowels in Spanish. The only way to remedy this problem is to fix it once it has been imported into Excel. 

Once in Excel, I renamed the worksheet *WuggyRawOutput*. Then I duplicated this worksheet and renamed the new worksheet *WuggyBuilt*. I then deleted all the columns except for columns A and B ("Word" and "Match"). I then added two columns called "Nonword_Concat" and "Rand".
In the *Nonword_Concat* column, I entered the following formulas:

* For CV-CV-CV words: =CONCATENATE(LEFT(B2,2),MID(B2,4,2),MID(B2,7,2))
    * This formula combines the two leftmost characters, skips character 3 (the first syllable boundary marker), adds the 4th and 5th characters to the string store, skips the 6th character (the second syllable boundary marker), and finally adds the 7th and 8th characters from the left to create the six-letter nonword.


* For CVC-CVC words: =CONCATENATE(LEFT(B403,3),RIGHT(B403,3))
    * This formula combines the three leftmost and three rightmost characters to create the six-letter nonword excluding the one and only syllable boundary marker.

These two formulas are necessary in order to create nonwords without the syllable markers. Once these steps have been completed, the file is ready for the python code housed in the cell below.

## Python Code

The python code here first imports the packages and functions needed to excute all the code correctly:

* pandas - an package to manipulate tabular data
* segFunctions - a function that creates csv files extracted from an Excel Workbook.

The script then calls segFunctions to create a temporary csv file that is then read in and stored as *df*. I then create an empty list to store the words I will extract with the script and setup three different counters. *i* is the counter that I will use reference the rows (minimum range value) in my dataframe and since python is zero-based, I set its default value to zero (0). *j* is the counter that I will use to reference the rows (maximum range value). **Note: this only works when the maximum number of candidates of 10 was actually reached when the Wuggy.app analysis ran** *This can be checked by multiplying the number of words submitted to Wuggy.app by 10. k* is the counter that will be used to reference the first row of the sorted dataframes and will only increase when duplicate nonword entries are found. Once the it has resolved the nonword duplicates it is reset to zero(0) for the next iteration.

Once the preamble is squared away and checks have been made to ensure datastructure is accurate, the meat of the code is executed. While *j* is less the 800, *df_temp* is stored as all columns from *df* with rows between *i* and *j*. Then *df_sorted* stores a new dataframe organized in ascending order according to the *Rand* column, which contains random numbers between 0 and 1, generated from Excel. Using the *df_sorted* dataframe, *token* stores the value from row *k*,zero(0) by default, and column 3. If the token is found in the list *nonwords*, it increases the value of *k* by one to go to the next row and tests it again. This process occurs until *token* is not found in the list. When *token* does not exist in the list, it appends the value of *token* to the list *nonwords* and then increases the values of *i and j* by 10 to get to the next set of data. This process repeats through the entire dataframe. In the current state, the length of the list *nonwords* should be 80. This can be checked using the following code snippet:

```len(nonwords)```

Following all the iterations, the list is added to a dataframe called *df_nonwords*, which is then passed through to a csv file stored in the *./processed_data/exp_files/* directory for later use.  


In [ ]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')

# reads in the csv created by csv_from_excel function
df = pd.read_csv('./data/temp_data/nonwords_all.csv')

nonwords = [] #creates an empty list for appending nonwords
i = 0 # counter used in row delimiter of dataframe
j = 9 # counter used in column delimiter of dataframe
k = 0 # counter used to iterate to next item when duplicate found

#iterate through the dataframe of 800 nonwords generated with Wuggy.app
try:
    while j < 800: # stop iteration before index value is exceeded
        df_temp = df.loc[i:j,:] # subset dataframe by rows with all columns
        df_sorted = df_temp.sort_values(by='Rand',ascending = True) # sort ascending by Rand column
        token = df_sorted.iat[k,3] # store the value of cell in dataframe
        if token in nonwords: # check to see if word is already in list
            k += 1 # when duplicate found go to next row value and try again
        else: # no duplicate found run this
            nonwords.append(token) # append the new token to the list 'nonwords'
            i += 10 #increase by 10 to get to next dataset
            j += 10 #increase by 10 to get to next dataset
            k = 0 #reset row indexer to zero(0) 
except IndexError: # check to see that dataframe length has not been exceeded
    print("Upper bound reached") # print debug message
finally: # once everything is completed, print the list
    # passes the list to a dataframe, which is then written to a csv file
    df_nonwords = pd.DataFrame(data={"col1": nonwords})
    df_nonwords.to_csv('./data/processed_data/exp_files/nonwords_list.csv',header=False)

In [ ]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')

# reads in the csv created by csv_from_excel function
df = pd.read_csv('./data/temp_data/nonwords_all.csv')
print(df)
nonwords = [] #creates an empty list for appending nonwords
i = 0 # counter used in row delimiter of dataframe
j = 9 # counter used in column delimiter of dataframe
k = 0 # counter used to iterate to next item when duplicate found

#iterate through the dataframe of 800 nonwords generated with Wuggy.app
try:
    while j < len(df): # stop iteration before index value is exceeded
        df_temp = df.loc[i:j,:] # subset dataframe by rows with all columns
        df_sorted = df_temp.sort_values(by='Rand',ascending = True) # sort ascending by Rand column
        token = df_sorted.iat[k,3] # store the value of cell in dataframe
        if token in nonwords: # check to see if word is already in list
            k += 1 # when duplicate found go to next row value and try again
        else: # no duplicate found run this
            nonwords.append(token) # append the new token to the list 'nonwords'
            i += 10 #increase by 10 to get to next dataset
            j += 10 #increase by 10 to get to next dataset
            k = 0 #reset row indexer to zero(0)
        print(j)
        
except IndexError: # check to see that dataframe length has not been exceeded
    print('Upper bound reached')

In [ ]:
set_test = set(df.columns.values[0])
print(set_test)

In [ ]:
print(df.columns.values[3])

In [ ]:
pd.unique(df.columns.values[0:3])


In [ ]:
set(df.columns)

In [ ]:
set(df.columns.values[3])

In [ ]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')

# reads in the csv created by csv_from_excel function
df = pd.read_csv('./data/temp_data/nonwords_all.csv')

uniquewords = [] #creates an empty list for appending nonwords
k = 0 # counter used to iterate to next item when duplicate found

#print(k)
while k < len(df): # stop iteration before index value is exceeded
    token = df.iat[k,0] # store the value of cell in dataframe
    if token in uniquewords: # check to see if word is already in list
        k += 1
        continue
    else: # no duplicate found run this
        uniquewords.append(token) # append the new token to the list 'nonwords'
        #print(k, token)
        k += 1 #reset row indexer to zero(0)
print(uniquewords)
print(len(uniquewords))

# Starting Fresh after Major Updates from Miquel
**Start date:** 8-10-2019

In [1]:
import sys
sys.path.append('../../scripts/')

In [7]:
import pandas as pd
import segFunctions

excelBook = 'Lexical_Access_Experimental_Item_Setup.xlsx'
sheetNameList = ['All_LexExperiment_Lists','All_LexPractice_Lists','Syllabification_Lists','Syllabification_Practice','LexTaleEsp','LexTaleEng','BasicLangProfile']
outFileList = ['expLists_all.csv','pracLists_all.csv','sylexpLists_all.csv','sylpracLists_all.csv','lexTaleListEsp.csv','lexTaleListEng.csv','sp_en_blp_trials.csv']
savePath = './data/temp_data/'
tempPath = './data/original_data/exp_files/'

i=0
for sheet in sheetNameList:
    saveFile = savePath + outFileList[i]
    tempDestination =  tempPath + outFileList[i]
    # runs the csv_from_excel function:
    # creates csv for segmentation experiment lists
    segFunctions.csv_from_excel(excelBook,sheetNameList[i],saveFile)
    segFunctions.fix_csv(saveFile,tempDestination,{'corrAns': lambda x: int(float(x))})
    i += 1

{'corrAns': <function <lambda> at 0x10fb79268>}
0 OrderedDict([('Word', 'seguro'), ('transcription', 'se.ˈɣu.ɾo'), ('corrAns', '1.0'), ('sylStruc', 'CV'), ('initialSylStress', 'no'), ('wordStatus', 'word'), ('source', 'CarreirasPerea2002'), ('freq_WPM', '670.8894230769231'), ('DerivedWord', 'NA'), ('Condition', 'A'), ('counter', '1.0'), ('numRevealed', '3.0'), ('masking', '######'), ('Length', '6.0'), ('PrimeCalc', 'seg###'), ('Prime', 'seg###'), ('matching', 'mismatch')])
1 OrderedDict([('Word', 'bonito'), ('transcription', 'bo.ˈni.to'), ('corrAns', '1.0'), ('sylStruc', 'CV'), ('initialSylStress', 'no'), ('wordStatus', 'word'), ('source', 'CarreirasPerea2002'), ('freq_WPM', '92.9326923076923'), ('DerivedWord', 'NA'), ('Condition', 'A'), ('counter', '2.0'), ('numRevealed', '3.0'), ('masking', '######'), ('Length', '6.0'), ('PrimeCalc', 'bon###'), ('Prime', 'bon###'), ('matching', 'mismatch')])
2 OrderedDict([('Word', 'molino'), ('transcription', 'mo.ˈli.no'), ('corrAns', '1.0'), ('sylS

{'corrAns': <function <lambda> at 0x11ef5af28>}
0 OrderedDict([('Word', 'módico'), ('corrAns', '1.0'), ('initialSylStress', 'yes'), ('sylStruc', 'CV'), ('wordStatus', 'word'), ('source', 'Alameda & Cuentos 1995'), ('freq_WPM', '0.07211538461538461'), ('numRevealed', '3.0'), ('masking', '######'), ('Length', '6.0'), ('PrimeCalc', 'mód###'), ('Prime', 'mód###'), ('matching', 'mismatch')])
1 OrderedDict([('Word', 'cúbico'), ('corrAns', '1.0'), ('initialSylStress', 'yes'), ('sylStruc', 'CV'), ('wordStatus', 'word'), ('source', 'Alameda & Cuentos 1995'), ('freq_WPM', '0.1923077'), ('numRevealed', '2.0'), ('masking', '######'), ('Length', '6.0'), ('PrimeCalc', 'cú####'), ('Prime', 'cú####'), ('matching', 'match')])
2 OrderedDict([('Word', 'cárcel'), ('corrAns', '1.0'), ('initialSylStress', 'yes'), ('sylStruc', 'CVC'), ('wordStatus', 'word'), ('source', 'Alameda & Cuentos 1995'), ('freq_WPM', '79.35096153846153'), ('numRevealed', '2.0'), ('masking', '######'), ('Length', '6.0'), ('PrimeCalc',

{'corrAns': <function <lambda> at 0x11efc20d0>}
0 OrderedDict([('Word', 'caldera'), ('corrSyl', 'CAL'), ('corrAns', '2.0'), ('leftKey', 'CAL'), ('rightKey', 'CA'), ('condition', 'AB')])
1 OrderedDict([('Word', 'cantera'), ('corrSyl', 'CAN'), ('corrAns', '2.0'), ('leftKey', 'CAN'), ('rightKey', 'CA'), ('condition', 'AB')])
2 OrderedDict([('Word', 'ciruela'), ('corrSyl', 'CI'), ('corrAns', '3.0'), ('leftKey', 'CIR'), ('rightKey', 'CI'), ('condition', 'AB')])
3 OrderedDict([('Word', 'culata'), ('corrSyl', 'CU'), ('corrAns', '3.0'), ('leftKey', 'CUL'), ('rightKey', 'CU'), ('condition', 'AB')])
4 OrderedDict([('Word', 'gustazo'), ('corrSyl', 'GUS'), ('corrAns', '2.0'), ('leftKey', 'GUS'), ('rightKey', 'GU'), ('condition', 'AB')])
5 OrderedDict([('Word', 'latencia'), ('corrSyl', 'LA'), ('corrAns', '3.0'), ('leftKey', 'LAT'), ('rightKey', 'LA'), ('condition', 'AB')])
6 OrderedDict([('Word', 'pericia'), ('corrSyl', 'PE'), ('corrAns', '3.0'), ('leftKey', 'PER'), ('rightKey', 'PE'), ('condition'

{'corrAns': <function <lambda> at 0x11f121e18>}
0 OrderedDict([('Order', '0.0'), ('Word', 'platery'), ('realWord', 'no'), ('corrAns', '4.0')])
1 OrderedDict([('Order', '0.0'), ('Word', 'denial'), ('realWord', 'yes'), ('corrAns', '1.0')])
2 OrderedDict([('Order', '0.0'), ('Word', 'generic'), ('realWord', 'yes'), ('corrAns', '1.0')])
3 OrderedDict([('Order', '1.0'), ('Word', 'mensible'), ('realWord', 'no'), ('corrAns', '4.0')])
4 OrderedDict([('Order', '2.0'), ('Word', 'scornful'), ('realWord', 'yes'), ('corrAns', '1.0')])
5 OrderedDict([('Order', '3.0'), ('Word', 'stoutly'), ('realWord', 'yes'), ('corrAns', '1.0')])
6 OrderedDict([('Order', '4.0'), ('Word', 'ablaze'), ('realWord', 'yes'), ('corrAns', '1.0')])
7 OrderedDict([('Order', '5.0'), ('Word', 'kermshaw'), ('realWord', 'no'), ('corrAns', '4.0')])
8 OrderedDict([('Order', '6.0'), ('Word', 'moonlit'), ('realWord', 'yes'), ('corrAns', '1.0')])
9 OrderedDict([('Order', '7.0'), ('Word', 'lofty'), ('realWord', 'yes'), ('corrAns', '1.0'

KeyError: 'corrAns'

In [8]:
!rm -f ./data/original_data/exp_files/sp_en_blp_trials.csv
# move this from the temp folder now because it does not have a corrAns column
!mv -f ./data/temp_data/sp_en_blp_trials.csv ./data/processed_data/exp_files/sp_en_blp_trials.csv

In [9]:
# creates list for all trial types

trialType = ['exp','sylexp','sylprac'] # list for trial type and naming ,'prac' removed for lexical experiment
condName = ['A','C'] # list for condition labels
pracsylRange = [0,9,10,19] # list for slicing syllable practice ranges
expsylRange = [0,47,48,95] # list for slicing syllable ranges
explexRange = [0,319,320,639] # list for slicing lexical decision ranges


# runs through each condition for practice and experimental list conditions
for trial in trialType:
    i = 0 # set counter for start syl and seg ranges
    j = 1 # set counter for end syl and seg ranges
    # Reads in the newly created csv files
    df = pd.read_csv('./data/original_data/exp_files/' + trial + 'Lists_all.csv')

    for cond in condName: # sets up loop to run through 4 conditions (ABCD)
        if trial == 'sylprac': # Tests for sylprac lists
            condition = df.loc[pracsylRange[i]:pracsylRange[j],:] # selects all rows for list
            condition.to_csv('./data/processed_data/exp_files/'+trial+'Cond'+cond+'.csv') # saves temp file for further work
            i += 2 # adds two to value to change range
            j += 2 # adds two to value to change range
        elif trial == 'sylexp': # test for syllable lists
            condition = df.loc[expsylRange[i]:expsylRange[j],:] # selects all rows for list
            condition.to_csv('./data/processed_data/exp_files/'+trial+'Cond'+cond+'.csv') # saves temp file for further work
            i += 2 # adds two to value to change range
            j += 2 # adds two to value to change range
        else: # captures both practice and real segmentation experiment lists
            condition = df.loc[explexRange[i]:explexRange[j],:] # selects all rows for list 
            condition.to_csv('./data/processed_data/exp_files/'+trial+'Cond'+cond+'.csv') # saves temp file for further work
            i += 2 # adds two to value to change range
            j += 2 # adds two to value to change range

In [ ]:
!pwd

In [10]:
!mv -f ./data/original_data/exp_files/lexTaleListEng.csv ./data/processed_data/exp_files/lexTaleListEng.csv
!mv -f ./data/original_data/exp_files/lexTaleListEsp.csv ./data/processed_data/exp_files/lexTaleListEsp.csv
!mv -f ./data/original_data/exp_files/pracLists_all.csv ./data/processed_data/exp_files/pracLists_all.csv

In [11]:
!rm -f ./data/original_data/exp_files/*.csv
!rm -f ./data/temp_data/*.csv